In [3]:
!pip install selenium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 985 kB 5.4 MB/s 
     |████████████████████████████████| 140 kB 39.4 MB/s 
     |████████████████████████████████| 358 kB 37.8 MB/s 
     |████████████████████████████████| 58 kB 5.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.12 which is incompatible.


In [4]:
!apt-get update

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [91.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Hit:14 http:

In [5]:
!apt install chromium-chromedriver

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  chromium-browser chromium-browser-l10n chromium-codecs-ffmpeg-extra
Suggested packages:
  webaccounts-chromium-extension unity-chromium-extension
The following NEW packages will be installed:
  chromium-browser chromium-browser-l10n chromium-chromedriver
  chromium-codecs-ffmpeg-extra
0 upgraded, 4 newly installed, 0 to remove and 47 not upgraded.
Need to get 91.4 MB of archives.
After this operation, 309 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-codecs-ffmpeg-extra amd64 104.0.5112.101-0ubuntu0.18.04.1 [1,162 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-browser amd64 104.0.5112.101-0ubuntu0.

In [6]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.common.by import By

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
import numpy as np
import pandas as pd
import re
from geopy.geocoders import Nominatim

pd.options.display.max_columns = 999

In [7]:
# Obtenemos los datos
precios = pd.read_csv('/content/drive/MyDrive/reto/reto_precios.csv')
postal_code = pd.read_csv("/content/drive/MyDrive/reto/postal_code.csv", encoding='latin', dtype={'Código Postal': str})

In [8]:
postal_code = postal_code[['Código Postal','Asentamiento']].copy()

In [9]:
# un vistazo al dataset
precios.head(2)

,main_name,subtitle,link,location,price,attributes,timestamp,id,address,since,description,vendor,lat,lon,price_mod,price_currency,since_period,since_value,days_on_site,amenities,age_in_years,bathrooms,cellars,num_floors,monthly_fee,apartments_per_floor,disposition,parking_lots,floor_situated,orientation,num_bedrooms,department_type,m2,final_price,price_square_meter
0,Departamento - Roma Norte,Departamento en venta,https://departamento.metroscubicos.com/MLM-139...,"Sinaloa 20 , Roma Sur, Roma Sur, Cuauhtémoc, D...",9000000 pesos,"127 m² construidos,2 recs.",2022-03-29,1,"Sinaloa 20 , Roma Sur",Publicado hace 5 días,Garden house con 2 habitaciones y dos baños c...,Investemx,19.421317,-99.164182,9000000.0,pesos,days,5,5.0,3.0,0.0,2.0,NaN,1.0,0 MXN,NaN,Contrafrente,2,NaN,NaN,2.0,NaN,127.0,9000000.0,70866.141732
1,Preventa Exclusiva · Terraloza Roma Norte ·,Departamento en venta,https://departamento.metroscubicos.com/MLM-139...,"Tonala, Roma Norte, Cuauhtémoc, Distrito Federal",4264000 pesos,"54 m² construidos,1 rec.",2022-03-29,2,Tonala,Publicado hace 7 días,Mezcla perfecta entre E S T I L O y C A L I D ...,Avitana,19.421029,-99.163541,4264000.0,pesos,days,7,7.0,2.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,1.0,NaN,54.0,4264000.0,78962.962963


## Selección de columnas relevantes

*   Algunas se eliminaron por que había otras columnas con el mismo valor pero más limpias
*   Otras por no tener suficientes valores para tener en cuanta más del 90% de los datos sin valor
*   Alguna más por tener un sólo valor y no ofrecer información





In [10]:
precios.columns

Index(['main_name', 'subtitle', 'link', 'location', 'price', 'attributes',
       'timestamp', 'id', 'address', 'since', 'description', 'vendor', 'lat',
       'lon', 'price_mod', 'price_currency', 'since_period', 'since_value',
       'days_on_site', 'amenities', 'age_in_years', 'bathrooms', 'cellars',
       'num_floors', 'monthly_fee', 'apartments_per_floor', 'disposition',
       'parking_lots', 'floor_situated', 'orientation', 'num_bedrooms',
       'department_type', 'm2', 'final_price', 'price_square_meter'],
      dtype='object')

In [11]:
cols = ['vendor',
       'days_on_site', 'amenities', 'age_in_years', 'bathrooms', 'cellars',
       'num_floors', 'monthly_fee', 'apartments_per_floor', 'disposition',
       'parking_lots', 'floor_situated', 'orientation', 'num_bedrooms',
       'department_type', 'm2', 'final_price', 'price_square_meter']

In [12]:
precios[cols].head(3)

,vendor,days_on_site,amenities,age_in_years,bathrooms,cellars,num_floors,monthly_fee,apartments_per_floor,disposition,parking_lots,floor_situated,orientation,num_bedrooms,department_type,m2,final_price,price_square_meter
0,Investemx,5.0,3.0,0.0,2.0,NaN,1.0,0 MXN,NaN,Contrafrente,2,NaN,NaN,2.0,NaN,127.0,9000000.0,70866.141732
1,Avitana,7.0,2.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,1.0,NaN,54.0,4264000.0,78962.962963
2,Ebranuncios,4.0,NaN,0.0,3.0,2.0,3.0,NaN,NaN,NaN,2,NaN,NaN,3.0,NaN,163.0,8093500.0,49653.374233


In [13]:
precios.price_currency.unique()

array(['pesos'], dtype=object)

In [14]:
# eliminamos disposition y orientation por que tienen muy pocos datos
# eliminamos price_currency por que sólo tiene 1 valor y no genera información al modelo
for col in ['department_type','floor_situated','apartments_per_floor','monthly_fee','price_currency','disposition', 'days_on_site', 'orientation']:
  print(col)
  display(precios[col].value_counts(), precios[col].value_counts().sum())

department_type


Loft    39
Name: department_type, dtype: int64

39

floor_situated


1.0    14
2.0     4
3.0     2
4.0     2
5.0     2
Name: floor_situated, dtype: int64

...

60.0     157
35.0     114
7.0       92
365.0     63
90.0      55
300.0     52
56.0      42
120.0     33
4.0       29
240.0     28
270.0     27
39.0      26
150.0     25
330.0     18
210.0     18
12.0      17
31.0      16
24.0      15
48.0      14
180.0     10
46.0      10
25.0      10
3.0        8
11.0       7
21.0       7
8.0        6
6.0        6
40.0       6
55.0       5
19.0       5
36.0       4
22.0       4
41.0       4
5.0        4
45.0       3
18.0       3
28.0       3
14.0       3
27.0       3
59.0       3
33.0       3
730.0      3
15.0       2
26.0       2
44.0       2
20.0       2
17.0       2
29.0       1
9.0        1
16.0       1
52.0       1
54.0       1
38.0       1
2.0        1
1.0        1
50.0       1
34.0       1
Name: days_on_site, dtype: int64

981

orientation


Poniente    3
Sur         2
Oeste       1
Norte       1
Oriente     1
Name: orientation, dtype: int64

8

## Obtener dirección

In [15]:
precios['lat_lon'] = precios['lat'].astype(str)+','+precios['lon'].astype(str)

In [16]:
precios['location_clean'] = precios.location.str.lower().str.extract(r'(roma norte|roma sur|monterrey)')

In [17]:
precios.location_clean.value_counts()

roma norte    503
roma sur      383
monterrey      51
Name: location_clean, dtype: int64

In [18]:
precios['lat_lon']

0      19.4213173,-99.1641821
1      19.4210293,-99.1635414
2      19.4078138,-99.1618723
3       19.404992,-99.1602428
4      19.4196927,-99.1560364
                ...          
976    19.4057767,-99.1610681
977    19.4057767,-99.1610681
978    19.4057767,-99.1610681
979    19.4148447,-99.1602414
980      19.4029284,-99.16806
Name: lat_lon, Length: 981, dtype: object

In [ ]:
geolocator.reverse("19.4213173,-99.1641821")[0].split(",")[-3].strip()

In [20]:
postal_code.loc[postal_code['Código Postal']=='6000', 'Asentamiento'].values[0].lower()

'centro (área 1)'

In [21]:
geolocator = Nominatim(user_agent="geoapiExercises")

for idx, row in precios.iterrows():
  if pd.isna(row['location_clean']):
    aux_code = str(int(re.findall("\d{5}", geolocator.reverse(row['lat_lon'])[0])[0]))
    try:
      aux_col = postal_code.loc[postal_code['Código Postal']==aux_code, 'Asentamiento'].values[0].lower()
    except:
      print('otro')
      print(aux_code, geolocator.reverse(row['lat_lon'])[0])
      aux_col = geolocator.reverse(row['lat_lon'])[0].split(",")[-3].strip()
    precios.loc[precios.index == idx ,'location_clean'] = aux_col

otro
3400 Oxxo, Calle Andalucía, Benito Juárez, Ciudad de México, 03400, México
otro
22534 Calle Jorullo, Unión, Tijuana, Municipio de Tijuana, Baja California, 22534, México
otro
3104 José María Rico, Colonia Del Valle Sur, Ciudad de México, 03104, México
otro
3103 Calle Concepción Beistegui, Benito Juárez, Ciudad de México, 03103, México


In [22]:
precios.location_clean.value_counts()

roma norte          531
roma sur            387
monterrey            51
Ciudad de México      3
cuauhtémoc            2
centro (área 4)       2
condesa               2
hipódromo             1
juárez                1
Baja California       1
Name: location_clean, dtype: int64

In [23]:
precios.loc[precios.location_clean == "juárez"]

,main_name,subtitle,link,location,price,attributes,timestamp,id,address,since,description,vendor,lat,lon,price_mod,price_currency,since_period,since_value,days_on_site,amenities,age_in_years,bathrooms,cellars,num_floors,monthly_fee,apartments_per_floor,disposition,parking_lots,floor_situated,orientation,num_bedrooms,department_type,m2,final_price,price_square_meter,lat_lon,location_clean
473,Venta Depto Roma,Departamento en venta,https://departamento.metroscubicos.com/MLM-134...,"Cuauhtémoc, Distrito Federal",4200000 pesos,"80 m² construidos,2 recs.",2022-03-29,478,NaN,Publicado hace 3 meses,"DEPARTAMENTO EN VENTA, BUENA UBICACION, BUENAS...",Consultores Patrimoniales,19.42968,-99.157615,4200000.0,pesos,months,3,90.0,NaN,0.0,1.0,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,2.0,NaN,80.0,4200000.0,52500.0,"19.4296796,-99.1576152",juárez


In [24]:
def mapear(x):
  if x=="roma norte":
    return x
  elif x=="roma sur":
    return x
  elif x=="monterrey":
    return 'borrar'
  elif x=="Baja California":
    return 'borrar'
  if (x=="juárez") | (x=="cuauhtémoc"):
    return "roma norte"
  else:
    return 'otro'

In [25]:
precios.location_clean = precios.location_clean.map(mapear)

In [26]:
precios.location_clean.value_counts(1)

roma norte    0.544343
roma sur      0.394495
borrar        0.053007
otro          0.008155
Name: location_clean, dtype: float64

In [27]:
precios = precios.loc[precios.location_clean != "borrar"].copy()
precios.location_clean.value_counts(1)

roma norte    0.574812
roma sur      0.416577
otro          0.008611
Name: location_clean, dtype: float64

## Dividir variables continuas, categóricas, target y limpieza

In [28]:
others = ['price_mod', 'm2','final_price']
cols = ['location_clean', 'days_on_site', 'amenities', 'age_in_years',
        'bathrooms', 'cellars', 'num_floors', 'monthly_fee', 'parking_lots',
        'num_bedrooms']
target = ['price_square_meter']
cols_cat = ['location_clean', 'days_on_site', 'amenities', 'age_in_years',
            'bathrooms', 'cellars', 'num_floors', 'monthly_fee', 'parking_lots',
            'num_bedrooms']
cols_con = ['days_on_site', 'age_in_years']

In [29]:
precios['monthly_fee'] = precios['monthly_fee'].str.replace(' MXN', '').fillna(0).astype(int)
precios['monthly_fee'] = precios['monthly_fee'].map(lambda x: 0 if x<=5 else 1)

In [30]:
precios['monthly_fee'].value_counts(1)

0    0.988159
1    0.011841
Name: monthly_fee, dtype: float64

In [31]:
precios.isnull().sum()

main_name                 0
subtitle                  0
link                      0
location                  0
price                     0
attributes                0
timestamp                 0
id                        0
address                 268
since                     0
description               0
vendor                    0
lat                       0
lon                       0
price_mod                 0
price_currency            0
since_period              0
since_value               0
days_on_site              0
amenities               462
age_in_years              0
bathrooms                 0
cellars                 877
num_floors              849
monthly_fee               0
apartments_per_floor    914
disposition             910
parking_lots              0
floor_situated          905
orientation             921
num_bedrooms              0
department_type         890
m2                        0
final_price               0
price_square_meter        0
lat_lon             

## Otro enfoque

In [32]:
cols = ['vendor','location_clean', 'days_on_site', 'amenities', 'age_in_years',
        'bathrooms', 'cellars', 'num_floors', 'monthly_fee',
        'apartments_per_floor', 'disposition',
        'parking_lots', 'floor_situated', 'orientation', 'num_bedrooms',
        'department_type', 'm2', 'final_price', 'price_square_meter']

In [33]:
precios.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 929 entries, 0 to 980
Data columns (total 37 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   main_name             929 non-null    object 
 1   subtitle              929 non-null    object 
 2   link                  929 non-null    object 
 3   location              929 non-null    object 
 4   price                 929 non-null    object 
 5   attributes            929 non-null    object 
 6   timestamp             929 non-null    object 
 7   id                    929 non-null    int64  
 8   address               661 non-null    object 
 9   since                 929 non-null    object 
 10  description           929 non-null    object 
 11  vendor                929 non-null    object 
 12  lat                   929 non-null    float64
 13  lon                   929 non-null    float64
 14  price_mod             929 non-null    float64
 15  price_currency        9

In [34]:
target = ['price_square_meter']
others = ['final_price']
cols_cat = ['location_clean', 'amenities',
            'bathrooms', 'cellars', 'num_floors', 'monthly_fee',
            'apartments_per_floor', 'disposition',
            'parking_lots', 'floor_situated', 'orientation', 'num_bedrooms',
            'department_type']
cols_con = ['days_on_site', 'age_in_years','m2']

In [35]:
for col in cols_cat:
  aux_mean = precios[[col, "price_square_meter"]].fillna("otro").groupby(col).agg({"price_square_meter": np.median})
  aux_count = precios[[col, "price_square_meter"]].fillna("otro").groupby(col).agg({"price_square_meter": "count"})
  display(aux_mean)
  display(aux_count)

,price_square_meter
location_clean,
otro,40057.692308
roma norte,62938.563373
roma sur,60625.000000


,price_square_meter
location_clean,
otro,8
roma norte,534
roma sur,387


,price_square_meter
amenities,
1.0,80017.857143
2.0,75057.975309
3.0,68809.523810
4.0,59403.606265
5.0,58059.701493
6.0,60020.221371
7.0,59878.980892
8.0,68945.113869
otro,62683.014027


,price_square_meter
amenities,
1.0,4
2.0,70
3.0,35
4.0,198
5.0,119
6.0,28
7.0,11
8.0,2
otro,462


,price_square_meter
bathrooms,
1.0,75099.919203
2.0,60254.906159
3.0,62189.735714
4.0,68083.333333
5.0,63372.093023
23.0,86527.472527


...

,price_square_meter
orientation,
Norte,1
Oeste,1
Oriente,1
Poniente,3
Sur,2
otro,921


,price_square_meter
num_bedrooms,
1.0,77250.294220
2.0,60624.186355
3.0,57798.404962
4.0,15669.550562


,price_square_meter
num_bedrooms,
1.0,136
2.0,568
3.0,224
4.0,1


,price_square_meter
department_type,
Loft,71563.217391
otro,61182.600511


,price_square_meter
department_type,
Loft,39
otro,890


In [36]:
precios.loc[precios.amenities == 1, "link"].values

array(['https://departamento.metroscubicos.com/MLM-1387297866-departamento-en-roma-norte-cuauhtemoc-con-1-recamara-52-m-id-60172-_JM#position=47&search_layout=grid&type=item&tracking_id=265bda14-7dfc-4b53-a670-dfe7b2108422',
       'https://departamento.metroscubicos.com/MLM-1387297846-departamento-en-roma-norte-cuauhtemoc-con-1-recamara-43-m-id-56554-_JM#position=23&search_layout=grid&type=item&tracking_id=9f6f369e-8e80-421b-ac8b-94248505a916',
       'https://departamento.metroscubicos.com/MLM-1387291490-departamento-en-roma-norte-cuauhtemoc-con-1-recamara-63-m-id-56590-_JM#position=16&search_layout=grid&type=item&tracking_id=53531504-9e7b-4b60-b195-ef526516c649',
       'https://departamento.metroscubicos.com/MLM-1387342906-departamento-en-roma-norte-cuauhtemoc-con-1-recamara-56-m-id-56589-_JM#position=17&search_layout=grid&type=item&tracking_id=53531504-9e7b-4b60-b195-ef526516c649'],
      dtype=object)

In [37]:
for i in cols:
  print(i)
  display(precios[i].value_counts())

vendor


Rent A House Cdmx            111
Metros Cuadrados Expertos    101
Buscatuhogarmexico            96
Bienesraiceslomelin           42
Houm                          39
                            ... 
Mexicosil                      1
Spacio Bienesraices            1
Enders Inmuebles               1
Constantino Bienes Raices      1
Aldo                           1
Name: vendor, Length: 123, dtype: int64

location_clean


roma norte    534
roma sur      387
otro            8
Name: location_clean, dtype: int64

days_on_site


...

78.0      27
111.0     23
86.0      22
90.0      22
93.0      22
          ..
7210.0     1
192.0      1
82.8       1
183.0      1
327.0      1
Name: m2, Length: 183, dtype: int64

final_price


4495000.0    9
7040000.0    8
2980000.0    8
4369000.0    8
6790000.0    7
            ..
5450689.0    1
5299915.0    1
5400000.0    1
5675313.0    1
6270000.0    1
Name: final_price, Length: 584, dtype: int64

price_square_meter


76410.256410    8
50000.000000    8
79750.000000    7
52267.441860    6
62189.735714    5
               ..
58499.887755    1
65789.473684    1
70329.000000    1
92644.030962    1
55000.000000    1
Name: price_square_meter, Length: 731, dtype: int64

In [38]:
precios[cols_con].describe()

,days_on_site,age_in_years,m2
count,929.000000,929.0,929.000000
mean,102.864370,0.0,112.914252
std,116.969515,0.0,237.753645
min,1.000000,0.0,31.000000
25%,31.000000,0.0,75.000000
50%,60.000000,0.0,93.000000
75%,120.000000,0.0,130.000000
max,730.000000,0.0,7210.000000


In [39]:
display(precios.loc[precios["m2"]<=75, "price_square_meter"].median())
display(precios.loc[(precios["m2"]>75)&(precios["m2"]<=90), "price_square_meter"].median())
display(precios.loc[(precios["m2"]>90)&(precios["m2"]<=130), "price_square_meter"].median())
display(precios.loc[precios["m2"]>130, "price_square_meter"].median())

68356.29534784464

59629.03540086271

62035.71428571428

54881.25

In [40]:
display(precios.loc[precios["days_on_site"]<=31, "price_square_meter"].median())
display(precios.loc[(precios["days_on_site"]>31)&(precios["days_on_site"]<=60), "price_square_meter"].median())
display(precios.loc[(precios["days_on_site"]>60)&(precios["days_on_site"]<=120), "price_square_meter"].median())
display(precios.loc[precios["days_on_site"]>120, "price_square_meter"].median())

63182.034909909904

61388.88888888889

61038.96103896104

60620.19230769231

## Intento Web Scrapping

In [12]:
wd = webdriver.Chrome('chromedriver', options=chrome_options)
wd.get('https://departamento.metroscubicos.com/MLM-1490896431-venta-de-departamento-en-el-campanario-_JM#position=-1&search_layout=grid&type=item&tracking_id=54b7c287-c207-4ec4-85f3-57da7616b164&BB=1')

In [13]:
html = wd.page_source
#element = wd.find_element(By.TAG_NAME , "table")

In [14]:
html

'<html lang="es-MX"><head><style type="text/css" class="drift-base-styles">.drift-bounding-box,.drift-zoom-pane{position:absolute;pointer-events:none}@keyframes noop{0%{zoom:1}}@-webkit-keyframes noop{0%{zoom:1}}.drift-zoom-pane.drift-open{display:block}.drift-zoom-pane.drift-closing,.drift-zoom-pane.drift-opening{animation:noop 1ms;-webkit-animation:noop 1ms}.drift-zoom-pane{overflow:hidden;width:100%;height:100%;top:0;left:0}.drift-zoom-pane-loader{display:none}.drift-zoom-pane img{position:absolute;display:block;max-width:none;max-height:none}</style><link rel="preconnect" href="https://www.google-analytics.com"><link rel="preconnect" href="https://www.google.com"><link rel="preconnect" href="https://data.mercadolibre.com"><link rel="preconnect" href="https://http2.mlstatic.com"><link rel="preconnect" href="https://stats.g.doubleclick.net"><link rel="preconnect" href="https://analytics.metroscubicos.com"><link rel="preconnect" href="https://analytics.metroscubicos.com"><link rel="pr